<a href="https://colab.research.google.com/github/Hongchenglong/colab/blob/main/CertifiableBayesianInference/Rayleigh_Experiments/analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import sys, os
from pathlib import Path
path = Path(os.getcwd())
sys.path.append(str(path.parent))
sys.path.append('/content/drive/MyDrive/CertifiableBayesianInference')
experiment = '/content/drive/MyDrive/ColabNotebooks/CertifiableBayesianInference/Rayleigh_Experiments/'

['', '/content', '/env/python', '/usr/lib/python37.zip', '/usr/lib/python3.7', '/usr/lib/python3.7/lib-dynload', '/usr/local/lib/python3.7/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.7/dist-packages/IPython/extensions', '/root/.ipython', '/', '/content/drive/MyDrive/CertifiableBayesianInference']


In [ ]:
!apt-get install tree
!tree /content/drive/MyDrive/ColabNotebooks/CertifiableBayesianInference/Rayleigh_Experiments/

/content/drive/MyDrive/ColabNotebooks/CertifiableBayesianInference/Rayleigh_Experiments/
├── analysis.ipynb
├── MNIST_runner.ipynb
└── SWAG_FCN_Posterior_5
    ├── arch.json
    ├── mean.npy
    ├── model.h5
    └── var.npy

1 directory, 6 files


In [ ]:
# import sys, os
# from pathlib import Path
# path = Path(os.getcwd())
# sys.path.append(str(path.parent))


import BayesKeras
from BayesKeras import PosteriorModel
from BayesKeras import analyzers

import tensorflow as tf
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

import numpy as np
from tqdm import trange


# import argparse
# parser = argparse.ArgumentParser()
# parser.add_argument("--opt")
# parser.add_argument("--rob")

# args = parser.parse_args()
# opt = str(args.opt)
# rob = int(args.rob)
opt = 'SWAG'
rob = 5

inference = opt
# 导入mnist数据集，划分训练集和测试集
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
# 像素强度除以255.0，将像素强度降低到0-1范围内
X_train = X_train/255.
X_test = X_test/255.
# 转换数据类型为float32，调整矩阵形状：行数未知，列数28*28
X_train = X_train.astype("float32").reshape(-1, 28*28)
X_test = X_test.astype("float32").reshape(-1, 28*28)

# 加载保存的后验分布模型
model = PosteriorModel(experiment + "%s_FCN_Posterior_%s"%(inference, rob))

# 交叉熵损失函数
# 稀疏分类交叉熵
loss = tf.keras.losses.SparseCategoricalCrossentropy()

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


11501568/11490434 [==============================] - 0s 0us/step
Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 512)               401920    
                                                                 
 dense_6 (Dense)             (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________
BayesKeras detected the above model 
 None


In [ ]:


num_images = 500
# 指标.准确率
accuracy = tf.keras.metrics.Accuracy()
# 返回后验预测分布的平均值：此函数对后验“n”次进行采样，并返回每个样本的平均 softmax 值
preds = model.predict(X_test[0:500]) #np.argmax(model.predict(np.asarray(adv)), axis=1)
# argmax: 返回沿轴的最大值的索引
accuracy.update_state(np.argmax(preds, axis=1), y_test[0:500])
fgsm = accuracy.result()
print("%s Accuracy: "%(inference), accuracy.result())

accuracy = tf.keras.metrics.Accuracy()
# 分析器.快速梯度标志方法 fast gradient sign method
adv = analyzers.FGSM(model, X_test[0:500], eps=0.1, loss_fn=loss, num_models=10)
preds = model.predict(adv) #np.argmax(model.predict(np.asarray(adv)), axis=1)
accuracy.update_state(np.argmax(preds, axis=1), y_test[0:500])
fgsm = accuracy.result()
print("FGSM Robustness: ", accuracy.result())

accuracy = tf.keras.metrics.Accuracy()
# 切尔诺夫界验证
preds = analyzers.chernoff_bound_verification(model, X_test[0:100], 0.1, y_test[0:100], confidence=0.80)
#print(preds.shape)
#print(np.argmax(preds, axis=1).shape)
accuracy.update_state(np.argmax(preds, axis=1), y_test[0:100])
print("Chernoff Lower Bound (IBP): ",  accuracy.result())

"""
p = 0
for i in trange(100, desc="Computing FGSM Robustness"):
    this_p = analyzers.massart_bound_check(model, np.asarray([X_test[i]]), 0.075, y_test[i])
    print(this_p)
    p += this_p
print("Massart Lower Bound (IBP): ", p/100.0)
"""


SWAG Accuracy:  tf.Tensor(0.98, shape=(), dtype=float32)
FGSM Robustness:  tf.Tensor(0.742, shape=(), dtype=float32)


Sampling for Chernoff Bound Satisfaction: 100%|██████████| 24/24 [00:00<00:00, 24.85it/s]

Chernoff Lower Bound (IBP):  tf.Tensor(0.25, shape=(), dtype=float32)


'\np = 0\nfor i in trange(100, desc="Computing FGSM Robustness"):\n    this_p = analyzers.massart_bound_check(model, np.asarray([X_test[i]]), 0.075, y_test[i])\n    print(this_p)\n    p += this_p\nprint("Massart Lower Bound (IBP): ", p/100.0)\n'

In [ ]:
!ls /content/

drive  sample_data
